In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import random
length=1648
plt.rcParams['axes.unicode_minus']=False
plt.rcParams['font.size'] = 15
plt.rcParams['font.family'] = 'Arial'
font={'family' : 'Arial',
        'color'  : 'black',
        'weight' : 'normal',
        'size'   : 15,
        }
length=1648
data_path='.../data2.xlsx'
save_path='...'
np.set_printoptions(precision=3) 

In [ ]:
def check_lithology(s1,s2):
    if s1==s2:
        return True
    else:
        return False
def check_borehole(x1,y1,x2,y2):
    if x1==x2:
        if y1==y2:
            return True
        else:
            return False
    else:
        return False
def check_up_down(z1,z2):
    if z1>z2:
        return 1  #1则表示former>latter
    else:
        return 0  #0则表示former<=latter
def weight_calc(x1,y1,z1,x2,y2,z2):
    weight=np.round((np.sqrt((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)),3)
    return weight
def check_distance(x1,y1,z1,x2,y2,z2,threshold):
    distance=np.round((np.sqrt((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)),3)
    if distance <= threshold:
        return True
    else:
        return False
def get_colors(name, lut):
    return plt.get_cmap(name, lut)([i for i in range(lut)])
color_list=get_colors('Reds',len(Es))
depth_list=get_colors('Blues',len(Es))

# Graph defination

In [ ]:
depth_unit=5
#由高指向低
#确定边
#确定权重
#定义geographical邻域
#定义stratigraphic邻域
strata_edges=[]
recorded_strata_number=[]
#分配strata_edges and strata_edges_weight 
for i in range(len(ID)):
    if ID[i] in recorded_strata_number:
        continue
    same_borehole_depth=[]
    node_number_list=[]
    same_borehole_depth.append(cord[i][2])
    node_number_list.append(ID[i])
    recorded_strata_number.append(ID[i])
    for j in range(len(ID)):
        if i==j:
            continue
        else:
            borehole_1_x=cord[i][0]
            borehole_2_x=cord[j][0]
            borehole_1_y=cord[i][1]
            borehole_2_y=cord[j][1]
        if check_borehole(borehole_1_x,borehole_1_y,borehole_2_x,borehole_2_y) == False:
            continue
        else:
            same_borehole_depth.append(cord[j][2])
            node_number_list.append(ID[j])
            recorded_strata_number.append(ID[j])
            
    borehole_depth,node_number=(list(t) for t in zip(*sorted(zip(same_borehole_depth,node_number_list))))
    for k in range(len(node_number)-1):
        edge_tmp=(node_number[k],node_number[k+1])
        weight_tmp=(node_number[k],node_number[k+1],
                  weight_calc(cord[node_number[k]][0],cord[node_number[k]][1],cord[node_number[k]][2],
                              cord[node_number[k+1]][0],cord[node_number[k+1]][1],cord[node_number[k+1]][2]))
        strata_edges.append(edge_tmp)

In [ ]:
threshold=200
#分配geo_edges 
geo_edges=[]
geo_weight_edges=[]
recorded_strata_number_2=[]
for i in range(len(ID)):
    if ID[i] in recorded_strata_number_2:
        continue
    same_lithology=[]
    same_lithology.append(ID[i])
    for j in range(len(ID)):
        if i==j:
            continue
        if check_lithology(soil[i],soil[j]) == False:
            continue
        else:
            same_lithology.append(ID[j])
    for k in range(len(same_lithology)):
        node_number_list2=[]
        node_number_list2.append(same_lithology[k])
        for l in range(len(same_lithology)):
            if k >= l:
                continue
            else:
                if check_distance(cord[same_lithology[k]][0],cord[same_lithology[k]][1],cord[same_lithology[k]][2],
                                  cord[same_lithology[l]][0],cord[same_lithology[l]][1],cord[same_lithology[l]][2],
                                  threshold) == True:
                    node_number_list2.append(same_lithology[l])
        recorded_strata_number_2.append(same_lithology[k])
        #建立全连接图的边
        for z1 in range(len(node_number_list2)-1):
            for z2 in range(len(node_number_list2)):
                if z1>=z2:
                    continue
                else:
                    edge_tmp=(node_number_list2[z1],node_number_list2[z2])
                    weight_tmp=(node_number_list2[z1],node_number_list2[z2],
                                weight_calc(cord[node_number_list2[z1]][0],cord[node_number_list2[z1]][1],cord[node_number_list2[z1]][2],
                                            cord[node_number_list2[z2]][0],cord[node_number_list2[z2]][1],cord[node_number_list2[z2]][2]))
                    geo_edges.append(edge_tmp)
                    geo_weight_edges.append(weight_tmp)

In [ ]:
#stratigraphic neighborhood
SGSPG_sn=nx.DiGraph()
SGSPG_sn.add_nodes_from(ID)
SGSPG_sn.add_edges_from(strata_edges)


#geographical neighborhood
SGSPG_gn=nx.Graph()
SGSPG_gn.add_nodes_from(ID)
SGSPG_gn.add_edges_from(geo_edges)

#geographical weight neighborhood
SGSPG_gn_w=nx.Graph()
SGSPG_gn_w.add_nodes_from(ID)
SGSPG_gn_w.add_weighted_edges_from(geo_weight_edges)

#SGSPG
SGSPG=nx.Graph()
SGSPG.add_nodes_from(ID)
SGSPG.add_edges_from(strata_edges)
SGSPG.add_edges_from(geo_edges)

# Visualization

In [ ]:
plt.figure(figsize=(10,10))
pos=nx.spring_layout(SGSPG_sn)
nx.draw_networkx(SGSPG_sn,node_size=40,pos=pos,node_color=color_list,with_labels=False)

In [ ]:
plt.figure(figsize=(12,12))
pos=nx.kamada_kawai_layout(SGSPG_gn)
nx.draw_networkx(SGSPG_gn,node_size=40,pos=pos,node_color=Es,with_labels=False,cmap='Reds',edge_color='darkgray')

In [ ]:
plt.figure(figsize=(12,12))
pos=nx.kamada_kawai_layout(SGSPG_gn)
nx.draw_networkx(SGSPG_gn,node_size=40,pos=pos,node_color=Z_D,with_labels=False,cmap='afmhot',edge_color='darkgray')

# Analysis

In [ ]:
degree_list_sgspg=[]
node_list_sgspg=[]
Es_list_sgspg=[]
depth_list_sgspg=[]
degree_list_sgspg_gn=[]
node_list_sgspg_gn=[]
Es_list_sgspg_gn=[]
depth_list_sgspg_gn=[]
degree_list_sgspg_sn=[]
node_list_sgspg_sn=[]
Es_list_sgspg_sn=[]
depth_list_sgspg_sn=[]
for node,degree in SGSPG.degree:
    node_list_sgspg.append(node)
    Es_list_sgspg.append(Es[node])
    degree_list_sgspg.append(degree)
    depth_list_sgspg.append(np.around(((Z_U[node]+Z_D[node])/2),3))
for node,degree in SGSPG_gn.degree:
    node_list_sgspg_gn.append(node)
    Es_list_sgspg_gn.append(Es[node])
    degree_list_sgspg_gn.append(degree)
    depth_list_sgspg_gn.append(np.around(((Z_U[node]+Z_D[node])/2),3))
for node,degree in SGSPG_sn.degree:
    node_list_sgspg_sn.append(node)
    Es_list_sgspg_sn.append(Es[node])
    degree_list_sgspg_sn.append(degree)
    depth_list_sgspg_sn.append(np.around(((Z_U[node]+Z_D[node])/2),3))

In [ ]:
plt.figure(figsize=(18,18))
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.subplots_adjust(left=0.1, bottom=0.1, right=1, wspace=0.3,hspace=0.3)
plt.subplot(3,3,1)
plt.bar(node_list_sgspg,degree_list_sgspg,color='dodgerblue')
plt.xlabel('Node ID',size=20)
plt.ylabel('Degree',size=20)
plt.subplot(3,3,2)
plt.bar(Es_list_sgspg,degree_list_sgspg,color='dodgerblue')
plt.xlim(4,12)
plt.xlabel('Es /(MPa)',size=20)
plt.ylabel('Max degree',size=20)
plt.subplot(3,3,3)
plt.bar(depth_list_sgspg,degree_list_sgspg,color='dodgerblue')
plt.xlabel('Depth /(m)',size=20)
plt.ylabel('Max degree',size=20)
plt.subplot(3,3,4)
plt.bar(node_list_sgspg_gn,degree_list_sgspg_gn,color='dodgerblue')
plt.xlabel('Node ID',size=20)
plt.ylabel('Degree',size=20)
plt.subplot(3,3,5)
plt.bar(Es_list_sgspg_gn,degree_list_sgspg_gn,color='dodgerblue')
plt.xlim(4,12)
plt.xlabel('Es /(MPa)',size=20)
plt.ylabel('Max degree',size=20)
plt.subplot(3,3,6)
plt.bar(depth_list_sgspg_gn,degree_list_sgspg_gn,color='dodgerblue')
plt.xlabel('Depth /(m)',size=20)
plt.ylabel('Max degree',size=20)
plt.subplot(3,3,7)
plt.bar(node_list_sgspg_sn,degree_list_sgspg_sn,color='dodgerblue')
plt.xlabel('Node ID',size=20)
plt.ylabel('Degree',size=20)
plt.subplot(3,3,8)
plt.bar(Es_list_sgspg_sn,degree_list_sgspg_sn,color='dodgerblue')
plt.xlim(4,12)
plt.xlabel('Es /(MPa)',size=20)
plt.ylabel('Max degree',size=20)
plt.subplot(3,3,9)
plt.bar(depth_list_sgspg_sn,degree_list_sgspg_sn,color='dodgerblue')
plt.xlabel('Depth /(m)',size=20)
plt.ylabel('Max degree',size=20)
plt.show()

In [ ]:
f=open(".../1.txt","w")
for Es in Es_list_sgspg:
    f.write(str(Es)+'\n')
f.close()
f=open(".../2.txt","w")
for degree in degree_list_sgspg:
    f.write(str(degree)+'\n')
f.close()

In [ ]:
#find max degree relation
Max_degree_list_sgspg=[]
Es_ID=[]
degree_tmp=[]
for i in range(len(Es_list_sgspg)):
    if len(Es_ID)==0:
        Es_ID_tmp=[]
        degree_tmp_tmp=[]
        for j in range(len(Es_list_sgspg)):
            if i==j:
                continue
            if Es_list_sgspg[i]==Es_list_sgspg[j]:
                Es_ID_tmp.append(node_list_sgspg[j])
        Es_ID_tmp.append(node_list_sgspg[i])
        degree_tmp_tmp.append(degree_list_sgspg[i])
        Es_ID.append(Es_ID_tmp)
        degree_tmp.append(degree_tmp_tmp)
    else:   
        check_list=[]
        for k in range(len(Es_ID)):
            for z in range(len(Es_ID[k])):
                check_list.append(Es_ID[k][z])
            
        if i not in Es_ID[k]:
            Es_ID_tmp=[]
            degree_tmp_tmp=[]
            for j in range(len(Es_list_sgspg)):
                if i>=j:
                    continue
                if Es_list_sgspg[i]==Es_list_sgspg[j]:
                    Es_ID_tmp.append(node_list_sgspg[j])
                    degree_tmp_tmp.append(degree_list_sgspg[j])
            Es_ID_tmp.append(node_list_sgspg[i])
            degree_tmp_tmp.append(degree_list_sgspg[j])
            Es_ID.append(Es_ID_tmp)
            degree_tmp.append(degree_tmp_tmp)
            
Es_max_degree_list_sgspg=[]
for i in range(len(degree_tmp)):
    for j in range(len(degree_tmp[i])):
        Max_degree_list_sgspg.append(max(degree_tmp[i]))
        Es_max_degree_list_sgspg.append(Es_list_sgspg[Es_ID[i][0]])

# Prediction performance analysis

In [ ]:
import torch
gt=torch.load('.../gt.pt')
pred=torch.load('.../pred.pt')

In [ ]:
error=list((pred-gt).numpy())
Es_list_sgspg_round4=list(np.round(Es_list_sgspg,4))
Es_list_gt=Es_list_sgspg_round4[999:1599]
depth_list_gt=depth_list_sgspg[999:1599]
degree_list_gt=degree_list_sgspg[999:1599]
max_degree_list_gt=Max_degree_list_sgspg[999:1599]

new_error,new_degree_list_gt=(list(t) for t in zip(*sorted(zip(error,degree_list_gt))))

In [ ]:
z1=np.polyfit(depth_list_gt,error,1)
p1=np.poly1d(z1)
y1=p1(depth_list_gt)
c1= (np.corrcoef(error, y1)[0,1])**2
z2=np.polyfit(max_degree_list_gt,error,1)
p2=np.poly1d(z2)
y2=p2(max_degree_list_gt)
c2= (np.corrcoef(error, y2)[0,1])**2
z3=np.polyfit(Es_list_gt,error,1)
p3=np.poly1d(z3)
y3=p3(Es_list_gt)
c3= (np.corrcoef(error, y3)[0,1])**2

new_z2=np.polyfit(new_degree_list_gt,new_error,1)
new_p2=np.poly1d(new_z2)
new_y2=new_p2(new_degree_list_gt)

plt.figure(figsize=(18,6))
plt.subplots_adjust(left=0.1, bottom=0.1, right=1, wspace=0.3,hspace=0.3)
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.subplot(1,3,1)
plt.scatter(new_degree_list_gt,new_error,color='dodgerblue')
plt.plot(new_degree_list_gt,new_y2,color='r',label=str(p2))
plt.title('R${^2}$'+'='+str(np.round(c2,5)))
plt.xlabel('Degree',size=20)
plt.ylabel('Residual /(MPa)',size=20)
plt.legend()
plt.subplot(1,3,2)
plt.scatter(Es_list_gt,error,color='dodgerblue')
plt.plot(Es_list_gt,y3,color='r',label=str(p3))
plt.xlabel('Es${^T}$ /(MPa)',size=20)
plt.ylabel('Residual /(MPa)',size=20)
plt.title('R${^2}$'+'='+str(np.round(c3,5)))
plt.legend()
plt.subplot(1,3,3)
plt.scatter(depth_list_gt,error,color='dodgerblue')
plt.plot(depth_list_gt,y1,color='r',label=str(p1))
plt.title('R${^2}$'+'='+str(np.round(c1,5)))
plt.xlabel('Depth /(m)',size=20)
plt.ylabel('Residual /(MPa)',size=20)
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(111)
plot1=ax1.plot(Es_max_degree_list_sgspg[999:1599],max_degree_list_gt,color='orange',label='Max degree')
ax2=ax1.twinx()
plot2=ax2.scatter(Es_max_degree_list_sgspg[999:1599],error,color='dodgerblue',label='Residual')
plot3=ax2.plot(Es_list_gt,y3,color='r',label=str(p3))
ax1.set_xlabel('Es${^T}$ /(MPa)',size=20)
ax2.set_ylabel('Residual /(MPa)',size=20,color='dodgerblue')
ax1.set_ylabel('Max degree',size=20,color='orange')

plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 20
#plt.grid(linestyle='-.',color='black')
for tl in ax1.get_yticklabels():
    tl.set_color('orange')
for tl in ax2.get_yticklabels():
    tl.set_color('dodgerblue')
for tl in ax1.get_xticklabels():
    tl.set_fontsize(16)
fig.legend(loc=1,fontsize=15,bbox_to_anchor=(0.57,0.89))
plt.title('R${^2}$'+'='+str(np.round(c3,5)))
plt.show()

In [ ]:
threshold_list=[0,1,10,100,200,400,600,800,1000,1200,1400,1600,1800,2000]
#geographical neighborhoo
def calc_average_connectivity_clustering_transitivity(ID,soil,threshold_list,save_path):
    Graph_list=[]
    ac_list=[]
    tt_list=[]

    for threshold in threshold_list:
        #分配geo_edges 
        geo_edges=[]
        geo_weight_edges=[]
        recorded_strata_number_2=[]
        degree_order_list=[]
        connectivity_list=[]
        for i in range(len(ID)):
            if ID[i] in recorded_strata_number_2:
                continue
            same_lithology=[]
            same_lithology.append(ID[i])
            for j in range(len(ID)):
                if i==j:
                    continue
                if check_lithology(soil[i],soil[j]) == False:
                    continue
                else:
                    same_lithology.append(ID[j])
            for k in range(len(same_lithology)):
                node_number_list2=[]
                node_number_list2.append(same_lithology[k])
                for l in range(len(same_lithology)):
                    if k >= l:
                        continue
                    else:
                        if check_distance(cord[same_lithology[k]][0],cord[same_lithology[k]][1],cord[same_lithology[k]][2],
                                          cord[same_lithology[l]][0],cord[same_lithology[l]][1],cord[same_lithology[l]][2],
                                          threshold) == True:
                            node_number_list2.append(same_lithology[l])
                recorded_strata_number_2.append(same_lithology[k])
                #建立全连接图的边
                for z1 in range(len(node_number_list2)-1):
                    for z2 in range(len(node_number_list2)):
                        if z1>=z2:
                            continue
                        else:
                            edge_tmp=(node_number_list2[z1],node_number_list2[z2])
                            weight_tmp=(node_number_list2[z1],node_number_list2[z2],
                                        weight_calc(cord[node_number_list2[z1]][0],cord[node_number_list2[z1]][1],cord[node_number_list2[z1]][2],
                                                    cord[node_number_list2[z2]][0],cord[node_number_list2[z2]][1],cord[node_number_list2[z2]][2]))
                            geo_edges.append(edge_tmp)
                            geo_weight_edges.append(weight_tmp)
        graph=nx.Graph()
        graph.add_nodes_from(ID)
        graph.add_edges_from(geo_edges)
        Graph_list.append(graph)
        ac=nx.average_clustering(graph)
        tt=nx.transitivity(graph)
        ac_list.append(ac)
        tt_list.append(tt)
        for degree,connectivity in enumerate(nx.average_degree_connectivity(graph)):
            degree_order_list.append(degree)
            connectivity_list.append(connectivity)
        plt.rcParams['xtick.labelsize'] = 18
        plt.rcParams['ytick.labelsize'] = 18
        plt.figure(figsize=(6,6))
        plt.bar(degree_order_list,connectivity_list,color='greenyellow')
        plt.xlabel('Degree',size=20)
        plt.ylabel('Connectivity',size=20)
        plt.title('$\it{T}$'+'='+str(threshold))
        plt.show()
        #plt.savefig(save_path+'SGSPG_threshold_'+str(threshold)+'.png',dpi=1000,bbox_inches='tight')
    return Graph_list,ac_list,tt_list

In [ ]:
Graph_list,ac_list,tt_list=calc_average_connectivity_clustering_transitivity(ID,soil,threshold_list,save_path_2)

In [ ]:
plt.figure(figsize=(45,13))
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, wspace=0.3,hspace=0.3)
for i in range(len(Graph_list)):
    threshold=threshold_list[i]
    plt.subplot(2,7,i+1)
    node_order_list=[]
    centrality_list=[]
    Es_order_list=Es
    for node,centrality in nx.degree_centrality(Graph_list[i]).items():
        node_order_list.append(node)
        centrality_list.append(centrality)
    plt.scatter(Es_order_list,centrality_list,label=threshold_list[i],color='darkorange')
    plt.title('$\it{T}$'+'='+str(threshold))
    plt.xlabel('Es /(MPa)',size=20)
    plt.ylabel('Centrality',size=20)
plt.show()

In [ ]:
R2_T_list=[0.83,0.94,0.95,0.98,0.99,
           0.991,0.992,0.994,0.995,0.995,
           0.995,0.995,0.995,0.995]

In [ ]:
fig=plt.figure(figsize=(8,8))
ax1=fig.add_subplot(111)
plot1=ax1.plot(threshold_list,ac_list,label='Clustering coefficient',marker='v',color='black')
plot2=ax1.plot(threshold_list,tt_list,label='Transitivity',marker='v',color='fuchsia')
plot3=ax1.plot(threshold_list,degree_assortativity_coefficient_list,label='Assortativity coefficient',marker='v',color='deepskyblue')
plot5=ax1.plot(threshold_list,R2_T_list,label='R${^2}$',marker='o',color='crimson')
ax2=ax1.twinx()
plot4=ax2.plot(threshold_list,graph_number_of_cliques_list,label='Cliques number',marker='s',color='gold')
ax2.set_ylabel('Cliques number',fontsize=18)
ax1.set_ylabel('Transiticity\nClustering coefficient\nAssortativity coefficient',fontsize=18)
ax1.set_xlabel('$\it{T}$',fontsize=18)
lines2=plot1+plot2+plot3+plot4+plot5
ax1.legend(lines2, [l.get_label() for l in lines2],loc='lower right',fontsize=18)
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 18
ax1.axhline(0.94,color='blueviolet',linestyle='--')
ax1.axvline(1,color='blue',linestyle='-.')
ax1.axhline(0.995,color='blueviolet',linestyle='--')
ax1.axvline(200,color='blue',linestyle='-.')
plt.show()

# community

In [ ]:
from networkx.algorithms import community
community_number_list=[]
community_list=[]
for i in range(len(Graph_list)-4):
    communities_generator = community.girvan_newman(Graph_list[i])
    top_level_communities = next(communities_generator)
    next_level_communities = next(communities_generator)
    sorted(map(sorted, next_level_communities))
    community_list.append(next_level_communities)
    community_number_list.append(len(list(next_level_communities)))
    print('Finished graph '+str(i+1)+', there is still extra '+str(len(Graph_list)-i-5))

In [ ]:
lithology=[]
recorder_soil=[]
for i in range(len(ID)):
    if ID[i] in recorder_soil:
        continue
    same_lithology=[]
    same_lithology.append(ID[i])
    for j in range(len(ID)):
        if i==j:
            continue
        if check_lithology(soil[i],soil[j]) == False:
            continue
        else:
            same_lithology.append(ID[j])
            recorder_soil.append(ID[j])
    lithology.append(same_lithology)

In [ ]:
from collections import OrderedDict
plt.figure("3D Scatter", facecolor="white",figsize=(8,8))
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['xtick.labelsize'] = 15
ax_1 = plt.gca(projection="3d")
label_lithology_list=[1,2,3,4,5,6,7,8,9,10,11,12,13,14]
for i in range(len(lithology)):
    for j in range(len(lithology[i])):
        ax_1.scatter(X[lithology[i][j]],Y[lithology[i][j]],-Z_D[lithology[i][j]],
                     label=str(label_lithology_list[i]),marker='o')
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(),bbox_to_anchor=(1.3,0.8),loc=1,fontsize=15)
#ax_1.set_xlabel('X')
#ax_1.set_ylabel('Y')
#ax_1.set_zlabel('Z')
plt.show()

# Heatmap

In [ ]:
# import difflib
import seaborn as sns
similarity_all=[]
for k in range(len(community_list)):
    similarity_of_node_list=np.zeros((len(lithology),len(list(community_list[k]))))
    for i in range(len(lithology)):
        for j in range(len(list(community_list[k]))):
            similarity_of_node_list[i][j]=difflib.SequenceMatcher(None,lithology[i],list(list(community_list[k])[j])).ratio()
    similarity_all.append(similarity_of_node_list)
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
for k in range(len(similarity_all)):
    if k!=9:
        continue
    threshold=threshold_list[k]
    plt.figure(figsize=(8,6))
    sns.heatmap(similarity_all[k],cmap='jet')
    #plt.title('$\it{T}$'+'='+str(threshold))
    plt.show()

# clique

In [ ]:
community_number_list_plot=[1518, 1479, 1210, 217, 129, 79, 48, 35, 32, 32, 32, 32, 32, 32]
plt.figure(figsize=(6,6))
plt.plot(threshold_list,community_number_list_plot,marker='s',color='gold')
plt.xlabel('$\it{T}$',fontsize=20)
plt.ylabel('Community number',fontsize=20)
plt.grid(axis='x')
plt.grid(axis='y')
plt.axhline(32,color='red',linestyle='--')
plt.text(700,100,'community number=32',fontsize=20,color='red')
plt.show()